# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686521


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:04<00:57,  2.06s/it]

Rendering models:  10%|█         | 3/30 [00:04<00:41,  1.53s/it]

Rendering models:  13%|█▎        | 4/30 [00:04<00:28,  1.11s/it]

Rendering models:  17%|█▋        | 5/30 [00:04<00:20,  1.21it/s]

...

Rendering models:  90%|█████████ | 27/30 [00:09<00:00,  6.24it/s]

Rendering models:  93%|█████████▎| 28/30 [00:10<00:00,  6.86it/s]

Rendering models:  97%|█████████▋| 29/30 [00:10<00:00,  7.41it/s]

Rendering models: 100%|██████████| 30/30 [00:10<00:00,  7.71it/s]

not-logged-in-20a275b53abec8db9f41    0.014610
not-logged-in-df8268e1657cfc2c5472    0.692867
Almudys                               0.001073
richiemac                             0.000319
kjkavene                              0.000339
not-logged-in-fd8b8ccf39505c820c25    0.000453
nsyee                                 0.000299
jmfranci                              0.000998
jfhood                                0.000345
tosnyder                              0.000325
coyotes54                             0.000309
JulieHedglin                          0.000276
acapirala                             0.000251
Lavadude                              0.016936
ShirleyL                              0.000278
bookert                               0.032936
FabioGalasso                          0.000379
nicoletraher                          0.005158
awesome_axolotl                       0.000674
not-logged-in-d5c52aed2534703878c3    0.255308
olmy.007                              0.000786
vceccare     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())